# 安装 AERO Suite

本文档以北[大未名一号高性能计算平台](https://hpc.pku.edu.cn/about.html)安装AERO Suite为例，进行讲解。


### 添加软件模块并创建环境变量

```shell
module purge
module load gcc/10.4.0
module load boost/1.83.0-gcc_10.4.0 
module load intel/2018.1
module load hdf5/1.8.19-intel-2018.1
module load netcdf/c/4.6.1-intel-2013.1

export HDF5DIR=/gpfs/share/software/hdf5/1.8.19/intel/2018.1
export NETCDF=/gpfs/share/software/netcdf/c/4.6.1/intel/2013.1
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HDF5DIR/lib:$NETCDF/lib
export BOOST_ROOT=/gpfs/share/software/boost/1.83.0/gcc_10.4.0 
``` 


### 检查以确保存在以下软件

```shell
which flex
which bison
echo $MKLROOT
echo $BOOST_ROOT
```

### 创建安装文件夹（frg）并创建环境变量
```shell
cd
mkdir frg
cd frg

export FRG_HOME=~/frg
```

### 下载安装Cmake并创建环境变量
```shell
curl -L -O https://github.com/Kitware/CMake/releases/download/v3.26.3/cmake-3.26.3-linux-x86_64.tar.gz
tar -xzvf cmake-3.26.3-linux-x86_64.tar.gz

export PATH=~/frg/cmake-3.26.3-linux-x86_64/bin:${PATH}
```


### 下载安装openmpi并创建环境变量
```shell
cd $FRG_HOME
curl -L -O  https://download.open-mpi.org/release/open-mpi/v4.0/openmpi-4.0.3.tar.gz
tar -xzvf openmpi-4.0.3.tar.gz
cd openmpi-4.0.3

# 在slurm集群系统进行配置
./configure \
--prefix=$FRG_HOME/openmpi-4.0.3/install \
--enable-mpi-interface-warning \
--enable-mpirun-prefix-by-default \
--without-verbs \
--enable-mpi-threads CC=gcc

# 并行安装
make -j 24 all
make install

export MPIDIR=$FRG_HOME/openmpi-4.0.3/install/bin
export PATH=$MPIDIR:${PATH}
```


### 下载安装eigen并创建环境变量
```shell
cd $FRG_HOME
curl -L -O https://gitlab.com/libeigen/eigen/-/archive/3.4.0/eigen-3.4.0.tar.gz
tar -xzvf eigen-3.4.0.tar.gz
cd eigen-3.4.0
mkdir build
cd build
cmake \
-DCMAKE_INSTALL_PREFIX=../install \
-DCMAKE_C_COMPILER=gcc \
-DCMAKE_CXX_COMPILER=g++ \
-DCMAKE_Fortran_COMPILER=gfortran \
..
make install
```

### 下载安装网格分割软件parmetis并创建环境变量
```shell
cd $FRG_HOME
## curl -L -O http://glaros.dtc.umn.edu/gkhome/fetch/sw/parmetis/parmetis-4.0.3.tar.gz
curl -L -O https://ftp.mcs.anl.gov/pub/pdetools/spack-pkgs/parmetis-4.0.3.tar.gz   
tar -xzvf parmetis-4.0.3.tar.gz
cd parmetis-4.0.3
make config prefix=$PWD/build cc=$FRG_HOME/openmpi-4.0.3/install/bin/mpicc cxx=$FRG_HOME/openmpi-4.0.3/install/bin/mpicxx
make install
cd metis
make config prefix=$PWD/build cc=$FRG_HOME/openmpi-4.0.3/install/bin/mpicc cxx=$FRG_HOME/openmpi-4.0.3/install/bin/mpicxx
make install

export PARMETISDIR=$FRG_HOME/parmetis-4.0.3/build
export METISDIR==$FRG_HOME/parmetis-4.0.3/metis/build
export METISPATH=$METISDIR
```

### 下载AERO Suite
```shell
cd $FRG_HOME
# Clone or pull (if already existing) latest frg repos
git clone https://bitbucket.org/frg/aero-f.git
git clone https://bitbucket.org/frg/aero-s.git
git clone https://bitbucket.org/frg/cd2tet.git
git clone https://bitbucket.org/frg/gmsh2top.git
git clone https://bitbucket.org/frg/matcher.git
git clone https://bitbucket.org/frg/metis.git
git clone https://bitbucket.org/frg/sower.git
git clone https://bitbucket.org/frg/xp2exo.git
```

### 下载安装流体力学软件aero-f并创建环境变量



#### 安装网格软件 CGAL 为各向异性的自适应网格
```shell
cd $FRG_HOME
git clone https://github.com/CGAL/cgal
cd cgal
mkdir build
cd build
cmake \
-DCMAKE_BUILD_TYPE=Release \
-DCMAKE_C_COMPILER=gcc \
-DCMAKE_CXX_COMPILER=g++ \
-DCMAKE_Fortran_COMPILER=gfortran \
-DCMAKE_INSTALL_PREFIX=./install .. 
make install
```

#### 安装几何工具箱 GeometricToolsEngine6p6 
```shell
cd $FRG_HOME
curl -L -O https://www.geometrictools.com/Downloads/GeometricToolsEngine6p6.zip
unzip GeometricToolsEngine6p6.zip
```

#### 安装流体力学软件aero-f
```shell
cd $FRG_HOME/aero-f
mkdir build
cd build

# 对aero-f进行系统配置
cmake \
-DCMAKE_C_COMPILER=gcc \
-DCMAKE_CXX_COMPILER=g++ \
-DCMAKE_Fortran_COMPILER=gfortran \
-DCMAKE_CXX_FLAGS="-march=znver2" \
-DCMAKE_C_FLAGS="-march=znver2" \
-DMPI_C_COMPILER=$MPIDIR/mpicc \
-DMPI_CXX_COMPILER=$MPIDIR/mpicxx \
-DMPI_Fortran_COMPILER=$MPIDIR/mpif90 \
-DBoost_INCLUDE_DIR:PATH=$BOOST_ROOT/include \
-DEIGEN3_INCLUDE_DIR:PATH=$FRG_HOME/eigen-3.4.0 \
-DWITH_EIGEN3:BOOL=true \
-DUSE_CONSISTENT_ORDERING:BOOL=ON \
-DUSE_MARCH_NATIVE:BOOL=false \
-DMETIS_INCLUDES:PATH=$FRG_HOME/parmetis-4.0.3/metis/build/include \
-DMETIS_LIBRARIES:FILEPATH=$FRG_HOME/parmetis-4.0.3/metis/build/lib/libmetis.a \
-DPARMETIS_INCLUDES:PATH=$FRG_HOME/parmetis-4.0.3/build/include \
-DPARMETIS_LIBRARIES:FILEPATH=$FRG_HOME/parmetis-4.0.3/build/lib/libparmetis.a \
-DLAPACK_FOUND:BOOL=true \
-DWITH_SCALAPACK=1 \
-DSCALAPACKLIB:FILEPATH="-Wl,--start-group ${MKLROOT}/lib/intel64/libmkl_scalapack_lp64.a ${MKLROOT}/lib/intel64/libmkl_intel_lp64.a ${MKLROOT}/lib/intel64/libmkl_sequential.a ${MKLROOT}/lib/intel64/libmkl_core.a ${MKLROOT}/lib/intel64/libmkl_blacs_intelmpi_lp64.a -Wl,--end-group -lpthread -lm -ldl" \
-DWITH_ARPACK=0 \
-DCMAKE_BUILD_TYPE=Release ${1:-.} \
-DWITH_CGAL:BOOL=ON \
-DCGAL_DIR:FILEPATH=$FRG_HOME/cgal/build \
-DWITH_GTE:BOOL=ON \
-DGTE_INCLUDE_DIR:FILEPATH=$FRG_HOME/GeometricTools/GTE \
-DMPIEXEC:FILEPATH=$FRG_HOME/openmpi-4.0.3/install/bin/mpiexec \
..

# 成功配置完毕过后输出是
# 
# =================================================
#           Summary of build options
# -------------------------------------------------
# Distributed execution:    YES
# Aeroelastic:              YES
# Embedded framework:       YES
# Adaptive repartitioning:  YES
# Modal:                    NO *******
# ScaLAPACK ROM:            YES
# Build type:               Release
# Extra link flags:
# =================================================


# 并行安装
make -j 24
```

### 下载安装固体力学软件aero-s

#### 安装数值线性代数软件 Trilinos
```shell
cd $FRG_HOME
git clone https://github.com/trilinos/Trilinos
cd Trilinos

# 对 Trilinos 进行系统配置
mkdir build
cd build
cmake \
-DCMAKE_INSTALL_PREFIX:FILEPATH=../install \
-DTPL_ENABLE_MPI:BOOL=ON \
-DTrilinos_ENABLE_ALL_PACKAGES:BOOL=OFF \
-DTrilinos_ENABLE_EXAMPLES:BOOL=OFF \
-DTrilinos_VERBOSE_CONFIGURE:BOOL=ON \
-DTrilinos_ENABLE_Zoltan:BOOL=ON \
-DZoltan_ENABLE_EXAMPLES:BOOL=OFF \
-DZoltan_ENABLE_TESTS:BOOL=OFF \
-DZoltan_ENABLE_ParMETIS:BOOL=OFF \
-DZoltan_ENABLE_Scotch:BOOL=OFF \
-DZoltan_ENABLE_PaToH:BOOL=OFF \
-DTrilinos_ASSERT_MISSING_PACKAGES:BOOL=OFF \
-D CMAKE_C_COMPILER=$MPIDIR/mpicc \
-D CMAKE_CXX_COMPILER=$MPIDIR/mpicxx \
-D CMAKE_Fortran_COMPILER=$MPIDIR/mpif90 \
.. |& tee OUTPUT.CMAKE

# 安装 Trilinos
make |& tee OUTPUT.MAKE
make install |& tee OUTPUT.INSTALL

```

#### 安装网格划分软件 scotch
```shell
cd $FRG_HOME
git clone https://gitlab.inria.fr/scotch/scotch.git

# 修改 makefile
cd scotch
cd src
cp Make.inc/Makefile.inc.x86-64_pc_linux2 ./Makefile.inc
# 修改 Makefile.inc 里面的:
# CCP             = mpicc
# 为 
# CCP             = $MPIDIR/mpicc

# 安装 scotch
make libscotch
```

#### 安装固体力学软件aero-s
```shell
cd $FRG_HOME/aero-s
mkdir build
cd build

# 对 aero-s 进行系统配置
cmake \
-DCMAKE_C_COMPILER=gcc \
-DCMAKE_CXX_COMPILER=g++ \
-DCMAKE_Fortran_COMPILER=gfortran \
-DMPI_C_COMPILER=$MPIDIR/mpicc \
-DMPI_CXX_COMPILER=$MPIDIR/mpicxx \
-DMPI_Fortran_COMPILER=$MPIDIR/mpif90 \
-DCMAKE_CXX_FLAGS="-march=znver2" \
-DCMAKE_C_FLAGS="-march=znver2" \
-DSCOTCH_DIR:PATH=$FRG_HOME/scotch \
-DBoost_INCLUDE_DIR:PATH=$BOOST_ROOT/include \
-DEigen3_DIR:PATH=$FRG_HOME/eigen-3.4.0/install/share/eigen3/cmake \
-DBLAS_FOUND:BOOL=TRUE \
-DBLAS_LIBRARIES:FILEPATH="" \
-DLAPACK_FOUND:BOOL=TRUE \
-DBLACS_FOUND:BOOL=TRUE \
-DBLACS_LIBRARIES:FILEPATH="" \
-DSCALAPACK_FOUND:BOOL=TRUE \
-DSCALAPACK_LIBRARY:FILEPATH="" \
-DLAPACK_LIBRARIES:FILEPATH="-Wl,--start-group ${MKLROOT}/lib/intel64/libmkl_scalapack_lp64.a ${MKLROOT}/lib/intel64/libmkl_intel_lp64.a ${MKLROOT}/lib/intel64/libmkl_sequential.a ${MKLROOT}/lib/intel64/libmkl_core.a ${MKLROOT}/lib/intel64/libmkl_blacs_intelmpi_lp64.a -Wl,--end-group -lpthread -lm -ldl" \
-DWITH_ARPACK=0 \
-DZOLTAN_INCLUDE_PATH=$FRG_HOME/Trilinos/install/include \
-DZOLTAN_SRC_PATH=$FRG_HOME/Trilinos/packages/zoltan/src \
-DZOLTAN_LIBRARY=$FRG_HOME/Trilinos/install/lib/libzoltan.a \
-DCMAKE_BUILD_TYPE=Release ${1:-.} \
-DMPIEXEC:FILEPATH=$MPIDIR/mpiexec \
-DEXTRALIB_MPI:FILEPATH=$MPIDIR/../lib/libmpi_mpifh.so \
..

# 成功配置完毕过后输出是
# =================================================
#           Summary of build options
# -------------------------------------------------
# MPI                       YES
# Distributed FETI:         YES
# Aeroelastic:              YES
# Mumps:                    NO
# Scalapack:                YES
# Arpack:                   NO
# Spooles:                  NO
# Cholmod:                  NO
# Pardiso:                  NO
# Acme:                     YES
# Metis:                    YES
# Eigen template library:   YES
# Torch:                    NO
# OpenMP:                   YES
# Build type:               Release
# Extra libraries:
# =================================================

# 并行安装
make -j 24
```

### 下载安装aero-s并行输出处理软件sower
```shell
cd $FRG_HOME/sower
mkdir build
cd build
cmake \
-DCMAKE_C_COMPILER=gcc \
-DCMAKE_CXX_COMPILER=g++ \
-DCMAKE_Fortran_COMPILER=gfortran \
..
make
```


### 下载安装aero-s网格处理软件cd2tet
```shell
cd $FRG_HOME/cd2tet
mkdir build
cd build
cmake \
-DCMAKE_C_COMPILER=gcc \
-DCMAKE_CXX_COMPILER=g++ \
-DCMAKE_Fortran_COMPILER=gfortran \
..
make
```


### 下载安装网格生成软件gmsh
```shell
cd $FRG_HOME
git clone https://gitlab.onelab.info/gmsh/gmsh.git

cd gmsh

mkdir build
cd build
cmake \
-DCMAKE_C_COMPILER=gcc \
-DCMAKE_CXX_COMPILER=g++ \
-DCMAKE_INSTALL_PREFIX=./ \
..
make
make install
```
安装好的binary文件在build/bin/gmsh或者build/gmsh。
安装gmsh也可以直接在[gmsh主页](http://gmsh.info)下载binary文件使用。

### 下载安装aero-f网格处理软件gmsh2top，用于转化gmsh网格和aero suite 使用网格
```shell
cd $FRG_HOME/gmsh2top
mkdir build
cd build
cmake \
-DCMAKE_C_COMPILER=gcc \
-DCMAKE_CXX_COMPILER=g++ \
-DCMAKE_Fortran_COMPILER=gfortran \
..
make
```



### 下载安装aero-suite网格匹配软件matcher，用于流固耦合模拟
```shell
cd $FRG_HOME/matcher
mkdir build
cd build
cmake \
-DCMAKE_C_COMPILER=gcc \
-DCMAKE_CXX_COMPILER=g++ \
-DCMAKE_Fortran_COMPILER=gfortran \
..
make
```



### 下载安装aero-s网格划分软件metis
```shell
cd $FRG_HOME/metis
mkdir build
cd build
cmake \
-DCMAKE_C_COMPILER=gcc \
-DCMAKE_CXX_COMPILER=g++ \
-DCMAKE_Fortran_COMPILER=gfortran \
..
make
```



### 下载安装aero-suite后处理软件xp2exo


#### 首先下载安装 Sandia 有限元后处理软件 seacas
```shell
cd $FRG_HOME
git clone https://github.com/gsjaardema/seacas.git

# 创建seacas使用的环境变量
cd seacas
export ACCESS=$PWD
export CC=gcc
export CXX=g++

# 修改 cmake-exodus 文件
# 把 40-42 行改为:
# NETCDF_PATH=${NETCDF}
# PNETCDF_PATH=${INSTALL_PATH}
# HDF5_PATH=${HDF5DIR}

mkdir build
cd build
../cmake-exodus
make
make install
```

#### 安装aero-suite后处理软件xp2exo
```shell
cd $FRG_HOME/xp2exo

mkdir build

cd build

cmake \
-DCMAKE_C_COMPILER=gcc \
-DCMAKE_CXX_COMPILER=g++ \
-DCMAKE_Fortran_COMPILER=gfortran \
-DNetCDF_INCLUDE_DIR=$NETCDF/include \
-DNetCDF_LIBRARIES="$NETCDF/lib/libnetcdf.so;$HDF5DIR/lib/libhdf5.so;$HDF5DIR/lib/libhdf5_hl.so" \
-DEXODUS_INCLUDE_DIR=$FRG_HOME/seacas/include \
-DEXODUS_LIBRARIES=$FRG_HOME/seacas/lib/libexodus.so \
..

make
```




# 安装aero-f自适应网格初始网格生产软件
```shell
cd $FRG_HOME
curl -L -O https://bitbucket.org/frg/aero-f/downloads/Meshtools.tgz
tar -xzvf Meshtools.tgz
cd Meshtools
make
```





### 最后设置添加路径
```shell
export PATH=$FRG_HOME/aero-f/build/bin:$PATH
export PATH=$FRG_HOME/aero-s/build/bin:$PATH
export PATH=$FRG_HOME/sower//build/bin:$PATH
export PATH=$FRG_HOME/metis/build/bin:$PATH
export PATH=$FRG_HOME/matcher/build/bin:$PATH
export PATH=$FRG_HOME/gmsh2top/build/bin:$PATH
export PATH=$FRG_HOME/cd2tet/build/bin:$PATH
export PATH=$FRG_HOME/xp2exo/build/bin:$PATH
export PATH=$FRG_HOME/Meshtools/bin:$PATH
export PATH=$FRG_HOME/gmsh/build/bin:$PATH
```

一般我们可以把环境变量和路径添加放在 .bashrc_frg 文件，每次使用AERO-Suite, 我们可以先 
```shell
source .bashrc_frg
```
本文档的所有材料在 `openmpi_build_PKU.txt` 和 `.bashrc_frg`里面。